In [108]:
import nest_asyncio
nest_asyncio.apply()

In [109]:
import pandas as pd
import numpy as np

In [110]:
import asyncio
from binance import BinanceSocketManager
from binance.client import Client

In [111]:
api_key = ''
api_secret = ''

client = Client(api_key,api_secret)

In [112]:
client.get_account()

{'makerCommission': 10,
 'takerCommission': 10,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'updateTime': 1644088397054,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '0.01703741', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '0.00000236', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'EOS', 'free': '6.10000000', 'locked': '0.00000000'},
  {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'GAS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BCC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '0.00000000', 'locked'

In [113]:
ST = 8
LT = 13
#The EMA of 8 n 13 on 15mins chart

In [114]:
def gethistoricals(symbol, LT):
    df = pd.DataFrame(client.get_historical_klines(symbol, '1d', str(LT) + 'days ago UTC', '1 day ago UTC'))
    #df = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback + ' min ago UTC'))
    #This is a data frame for the symbol, timeframe, lookback period of the bigger MA, until when time so lookback till when, from when to when
    
    closes = pd.DataFrame(df[4])
    #Getting just the 4th column of the data, that's the Close column
    closes.columns = ['Close']
    closes['ST'] = closes.Close.rolling(ST-1).sum()
    closes['LT'] = closes.Close.rolling(LT-1).sum()
    closes.dropna(inplace=True)
    return closes
    

In [115]:
historicals = gethistoricals('DOGEUSDT', LT)
historicals

,Close,ST,LT
11,0.15390000,1.0073,1.7153


In [121]:
historicals['ST'] = 1.75

In [122]:
def liveSMA(hist, live): #Getting EMA of historical data n live data
    liveST = (hist['ST'].values + live.Price.values) / ST
    liveLT = (hist['LT'].values + live.Price.values) / LT
    
    return liveST, liveLT
    

In [123]:
def createframe(msg):#In general this is to get a msg from binance n make it readable, make sense n workable
    df = pd.DataFrame([msg])#The msg as a dataframe
    df = df.loc[:,['s','E','p']]# Just these 3 columns needed
    df.columns = ['symbol', 'Time', 'Price'] #Renaming the 3 columns
    df.Price = df.Price.astype(float) #Converting the prices to float(calculable) from string
    df.Time = pd.to_datetime(df.Time, unit='ms')#Convert the time index to a datetime index
    return df


In [124]:
async def strat(coin, qty, SL_limit, open_position = False):#Now creating our strategy
    bm = BinanceSocketManager(client)#Variables for the binance socket manager
    ts = bm.trade_socket(coin)#trade socket manager for the coin we want to trade
    async with ts as tscm:
        while True:
            res = await tscm.recv() #While loop to continue receiving msgs from binance
            if res: #This step is just to confirm that we are actually getting msgs from the socket(binance)
                frame = createframe(res)
                print(frame)
                
                livest, livelt = liveSMA(historicals,frame)
                if livest > livelt and not open_position:
                    #If Smaller SMA is above Bigger SMA then open postion if not already opened position
                    order = client.create_order(symbol=coin, side='BUY', type='MARKET', quantity=qty)
                    print(order)
                    
                    buyprice = float(order['fills'][0]['price']) #Stop loss, this is just filtering out for the price
                    open_position = True
                if open_position: #Selling conditions, first check if there's an open position
                    if frame.Price[0] < buyprice * SL_limit or frame.Price[0] > 1.02 * buyprice:
                    #Sell if price is below the buy price times the stop loss limit eg 98% or price is above 2% (1.02) times the buyprice
                        order = client.create_order(symbol=coin, side='SELL', type='MARKET', quantity=qty)
                        print(order)
                    
                    loop.stop()#This will stop the live stream too n shut down the whole kernel
                    
                    

In [126]:
info = client.get_symbol_info('DOGEUSDT')
print(info)

{'symbol': 'DOGEUSDT', 'status': 'TRADING', 'baseAsset': 'DOGE', 'baseAssetPrecision': 8, 'quoteAsset': 'USDT', 'quotePrecision': 8, 'quoteAssetPrecision': 8, 'baseCommissionPrecision': 8, 'quoteCommissionPrecision': 8, 'orderTypes': ['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_LIMIT', 'TAKE_PROFIT_LIMIT'], 'icebergAllowed': True, 'ocoAllowed': True, 'quoteOrderQtyMarketAllowed': True, 'isSpotTradingAllowed': True, 'isMarginTradingAllowed': True, 'filters': [{'filterType': 'PRICE_FILTER', 'minPrice': '0.00010000', 'maxPrice': '1000.00000000', 'tickSize': '0.00010000'}, {'filterType': 'PERCENT_PRICE', 'multiplierUp': '5', 'multiplierDown': '0.2', 'avgPriceMins': 5}, {'filterType': 'LOT_SIZE', 'minQty': '1.00000000', 'maxQty': '9000000.00000000', 'stepSize': '1.00000000'}, {'filterType': 'MIN_NOTIONAL', 'minNotional': '10.00000000', 'applyToMarket': True, 'avgPriceMins': 5}, {'filterType': 'ICEBERG_PARTS', 'limit': 10}, {'filterType': 'MARKET_LOT_SIZE', 'minQty': '0.00000000', 'maxQty':

In [130]:
if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(strat('DOGEUSDT', 100.0, 0.98))

     symbol                    Time   Price
0  DOGEUSDT 2022-02-07 07:07:36.298  0.1575


BinanceAPIException: APIError(code=-2010): Account has insufficient balance for requested action.